### Importamos las librerias necesarias

In [2]:
import os
import datetime

import requests
import lxml.html as html

from tqdm.notebook import tqdm

### Definimos como constante las declaraciones XPath que ya hayamos construido de nuestra página objetivo

In [3]:
HOME_URL = 'https://www.larepublica.co/'

XPATH_LINK_TO_ARTICLE = '//text-fill[not(@class)]/a/@href'    # Se cambio h2 por text-fill para que jalara el parsed

XPATH_TITLE = '//div[@class="mb-auto"]/h2/span/text()'    # Se cambio h2 por text-fill para que jalara el parsed

XPATH_SUMMARY = '//div[@class="lead"]/p/text()'

XPATH_BODY = '//div[@class="html-content"]/p[not(@class)]/text()'

### Definimos la función que parseará el home de la página y obtendrá los links de los artículos

In [4]:
def parse_home():
    # Procuremos envolver la petición en un bloque try-except
    try:
        # Realizamos la petición al sitio web
        print(f"Realizamos la petición a {HOME_URL}")
        response = requests.get(HOME_URL)
        
        if response.status_code == 200:
            # Decodificamos el contenido de la respuesta como utf-8
            home = response.content.decode('utf-8')
            
            # Parsearemos el string respuesta a un HTML con superpoderes
            parsed = html.fromstring(home)
            
            # Obtenemos los links a partir de la instrucción XPath previamente probada
            links_to_notices = parsed.xpath(XPATH_LINK_TO_ARTICLE)
            
            # Obtenemos la fecha de hoy
            today = datetime.date.today().strftime('%d-%m-%Y')
            
            # Creamos una carpeta con la fecha de hoy para guardar las notas
            if not os.path.isdir(today):
                os.mkdir(today)

            # Iteramos sobre todos los links para obtener la nota completa
            for link in tqdm(links_to_notices):
                parse_notice(link, today)
        
        else:
            raise ValueError(f'Error: {response.status_code}')
            
    except ValueError as ve:
        print(ve)

### Definimos la función que obtendrá la información de cada uno de los links

In [13]:
def parse_notice(link, today):
    try:
        # Hacemos la petición al link de la noticia
        response = requests.get(link)
        if response.status_code == 200:
            notice = response.content.decode('utf-8')
            parsed = html.fromstring(notice)
            
            # Tratamos de extraer la información del Título, resumen y el cuerpo de la nota.
            try:
                title = parsed.xpath(XPATH_TITLE)[0]
                title = title.replace('\"', '').replace(',', '').strip()
                summary = parsed.xpath(XPATH_SUMMARY)[0]
                body = parsed.xpath(XPATH_BODY)
                    
            except IndexError as ie:
                return
            
            # Escribimos cada una de las notas en un archivo .txt 
            with open(f'{today}/{title}.txt', 'w', encoding='utf-8') as f:
                f.write(title)
                f.write('\n\n')
                f.write(summary)
                f.write('\n\n')

                for p in body:
                    f.write(p)
                    f.write('\n')

                f.write('\n\n\n')
                f.write(f'Link de la noticia: {link}')
        else:
            raise ValueError(f'Error: {response.status_code}')
            
    except ValueError as ve:
        print(ve)

### Ejecutamos el script en un entry-point

In [14]:
if __name__ == "__main__":
    parse_home()

Realizamos la petición a https://www.larepublica.co/


  0%|          | 0/78 [00:00<?, ?it/s]